### 1. 安装HuggingFace 并下载模型到本地

In [1]:
region = "us-west-2"
from sagemaker import image_uris
ecr_image = image_uris.retrieve(
    framework="huggingface",
    region=region,
    version="4.6.1",
    image_scope="inference",
    base_framework_version="pytorch1.7.1",
    py_version="py36",
    container_version="ubuntu18.04",
    instance_type="ml.m5.xlarge",
)
ecr_image

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-inference:1.7.1-transformers4.6.1-cpu-py36-ubuntu18.04'

In [2]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

  Using cached sagemaker-2.233.0-py3-none-any.whl.metadata (16 kB)
Using cached sagemaker-2.233.0-py3-none-any.whl (1.6 MB)
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.232.3
    Uninstalling sagemaker-2.232.3:
      Successfully uninstalled sagemaker-2.232.3


In [3]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./bge-reranker-v2-m3")
local_model_path.mkdir(exist_ok=True)
model_name = "BAAI/bge-reranker-v2-m3"
commit_hash = "12e974610ba9083ed95f3edf08d7e899581f4de4"

In [4]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

assets/BEIR-bge-en-v1.5.png:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

assets/CMTEB-retrieval-bge-zh-v1.5.png:   0%|          | 0.00/51.5k [00:00<?, ?B/s]

assets/miracl-bge-m3.png:   0%|          | 0.00/52.0k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

'bge-reranker-v2-m3/models--BAAI--bge-reranker-v2-m3/snapshots/12e974610ba9083ed95f3edf08d7e899581f4de4'

### 2. 把模型拷贝到S3为后续部署做准备

In [5]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [6]:
s3_model_prefix = f"aigc-llm-models/{model_name}"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = f"aigc-llm-models/{model_name}_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: aigc-llm-models/BAAI/bge-reranker-v2-m3_deploy_code
model_snapshot_path: bge-reranker-v2-m3/models--BAAI--bge-reranker-v2-m3/snapshots/12e974610ba9083ed95f3edf08d7e899581f4de4


In [7]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: bge-reranker-v2-m3/models--BAAI--bge-reranker-v2-m3/snapshots/12e974610ba9083ed95f3edf08d7e899581f4de4/.gitattributes to s3://sagemaker-us-west-2-236995464743/aigc-llm-models/BAAI/bge-reranker-v2-m3/.gitattributes
upload: bge-reranker-v2-m3/models--BAAI--bge-reranker-v2-m3/snapshots/12e974610ba9083ed95f3edf08d7e899581f4de4/README.md to s3://sagemaker-us-west-2-236995464743/aigc-llm-models/BAAI/bge-reranker-v2-m3/README.md
upload: bge-reranker-v2-m3/models--BAAI--bge-reranker-v2-m3/snapshots/12e974610ba9083ed95f3edf08d7e899581f4de4/assets/BEIR-bge-en-v1.5.png to s3://sagemaker-us-west-2-236995464743/aigc-llm-models/BAAI/bge-reranker-v2-m3/assets/BEIR-bge-en-v1.5.png
upload: bge-reranker-v2-m3/models--BAAI--bge-reranker-v2-m3/snapshots/12e974610ba9083ed95f3edf08d7e899581f4de4/assets/CMTEB-retrieval-bge-zh-v1.5.png to s3://sagemaker-us-west-2-236995464743/aigc-llm-models/BAAI/bge-reranker-v2-m3/assets/CMTEB-retrieval-bge-zh-v1.5.png
upload: bge-reranker-v2-m3/models--BAAI--bge-rer

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [8]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
if region in ['cn-north-1', 'cn-northwest-1']:
    inference_image_uri = (
        f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
    )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [9]:
local_code_dir = s3_code_prefix.split('/')[-1]
!mkdir -p {local_code_dir}

In [10]:
%%writefile {local_code_dir}/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from FlagEmbedding import FlagReranker

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    model = FlagReranker(model_location, use_fp16=True)
    
    return model

model = None

def handle(inputs: Input):
    global model 
    if not model:
        model  = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    queries = data["inputs"]
    docs = data["docs"]
    
    pairs = []
    for i,q in enumerate(queries):
        pairs.append([q,docs[i]])
    
    scores = model.compute_score(pairs)
      
    result = {"scores": scores}

    return Output().add_as_json(result)

Overwriting bge-reranker-v2-m3_deploy_code/model.py


In [11]:
s3_path = f"s3://{bucket}/{s3_model_prefix}/"
print(f"option.s3url ==> {s3_path}")

option.s3url ==> s3://sagemaker-us-west-2-236995464743/aigc-llm-models/BAAI/bge-reranker-v2-m3/


#### Note: option.s3url 需要按照自己的账号进行修改

In [12]:
%%writefile {local_code_dir}/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = S3PATH

Overwriting bge-reranker-v2-m3_deploy_code/serving.properties


In [13]:
!sed -i "s|option.s3url = S3PATH|option.s3url = {s3_path}|" {local_code_dir}/serving.properties

In [14]:
%%writefile {local_code_dir}/requirements.txt
FlagEmbedding

Overwriting bge-reranker-v2-m3_deploy_code/requirements.txt


In [15]:
!rm model.tar.gz
!cd {local_code_dir} && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz {local_code_dir}

bge-reranker-v2-m3_deploy_code/
bge-reranker-v2-m3_deploy_code/model.py
bge-reranker-v2-m3_deploy_code/serving.properties
bge-reranker-v2-m3_deploy_code/requirements.txt


In [16]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-236995464743/aigc-llm-models/BAAI/bge-reranker-v2-m3_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [17]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("bge-reranker-v2-m3")# name_from_base("st-paraphrase-mpnet-base-v2") Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bge-reranker-v2-m3-2024-11-11-09-56-36-256
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-west-2:236995464743:model/bge-reranker-v2-m3-2024-11-11-09-56-36-256


#### 推理机型选择 (https://aws.amazon.com/cn/sagemaker/pricing/)
- GPU
  + ml.g4dn.xlarge 按需价格 0.526 USD/Hour
- CPU
  + ml.c5.xlarge   按需价格 0.204 USD/Hour

In [18]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:236995464743:endpoint-config/bge-reranker-v2-m3-2024-11-11-09-56-36-256-config',
 'ResponseMetadata': {'RequestId': 'a4816a8e-b2ea-45e4-acd7-32ca524a20e5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a4816a8e-b2ea-45e4-acd7-32ca524a20e5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '130',
   'date': 'Mon, 11 Nov 2024 09:56:36 GMT'},
  'RetryAttempts': 0}}

In [19]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")


KeyboardInterrupt



In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### 5. 模型测试

In [22]:
def rerank_by_sm_endpoint(questions, docs, sm_client, endpoint_name):
    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "docs": docs
            }
        ),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    scores = [item for item in json_obj['scores']]
    return scores

In [23]:
prompts1 = ["请问AWS Clean Rooms是多方都会收费吗？"] * 2
docs1 = ["会收费","生成式AI(generative AI/Gen AI)是一种AI技术,可以创造新的内容和想法的人工智能，例如图像、视频、文本、代码、音乐等。它利用机器学习模型基于大量数据进行预训练得到的超大模型也即基础模型来提供支持。"]
endpoint_name = "bge-reranker-v2-m3"
print(rerank_by_sm_endpoint(prompts1, docs1, smr_client, endpoint_name))

[-1.6298828125, -11.0390625]
